In [ ]:
import pylab
import numpy as np
np.seterr(all='raise')
np.random.seed(13)
import scipy as sp
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
# mpl.rc('text', usetex=True)
%matplotlib inline
from tqdm import tqdm

from sklearn import datasets

In [ ]:
def calc_error(res,e,names=[]):
    tmp_i = np.zeros(len(res.x))
    etol = e * max(1, abs(res.fun))
    for i in range(len(res.x)):
        tmp_i[i] = 1.0
        hess_inv_i = res.hess_inv(tmp_i)[i]
        uncertainty_i = np.sqrt(etol * hess_inv_i)
        tmp_i[i] = 0.0
        if len(names) > 0:
            print('{0} = {1:12.4e} ± {2:.1e}'.format(names[i], res.x[i], uncertainty_i))
        else:
            print('x^{0} = {1:12.4e} ± {2:.1e}'.format(i, res.x[i], uncertainty_i))

# Revisar bases de datos conocidas
[SciKit Learn](https://scikit-learn.org/stable/datasets/index.html)
<!-- X,Y = datasets.load_boston(return_X_y=True) -->

In [ ]:
datasets.load_boston()

In [ ]:
A = pd.DataFrame(datasets.load_boston().data, columns=datasets.load_boston().feature_names)
b = pd.DataFrame(datasets.load_boston().target, columns=["MEDV"])

In [ ]:
A

$$X = x_1+x_2+x_3+\dots$$
Mi intención,
$$CRIM\times x_1+ZN\times x_2+INDUS\times x_3+\dots = b$$

In [ ]:
Anumpy = np.matrix(A.to_numpy(dtype=np.float64))
bnumpy = np.matrix(b.to_numpy(dtype=np.float64))

In [ ]:
# %%timeit
X_opt = np.linalg.inv(Anumpy.T*Anumpy)*Anumpy.T*bnumpy

In [ ]:
print(X_opt.T)

In [ ]:
sns.scatterplot(np.array(Anumpy*X_opt).flatten(),np.array(bnumpy).flatten())
plt.plot([0,45],[0,45])

In [ ]:
predicciones = np.array(Anumpy*X_opt).flatten()

In [ ]:
observaciones = np.array(bnumpy).flatten()
bbarrita = observaciones.mean()

In [ ]:
r2 = 1-(np.linalg.norm(predicciones - observaciones)**2)/(np.linalg.norm(observaciones - bbarrita)**2)

In [ ]:
r2

# Implementemos nuestro algoritmo de Optimización por Newton Raphson
La idea es encontrar $X_{opt}$ de forma iterativa sin invertir la matriz

In [ ]:
# Definamos la func de costo (debe devolver un escalar)
def L(x,A,b):
    # (b_pred-b_obs)^2
    # m es el numero de datos
    # n es el numero de parametros == 13
    m,n = A.shape
    X = np.matrix(x).T
    DeltaB=(A*X-b)
    return (DeltaB.T*DeltaB)[0,0]/m
def dLdx(x,A,b):
    # (b_pred-b_obs)^2
    # m es el numero de datos
    # n es el numero de parametros == 13
    m,n = A.shape
    X = np.matrix(x).T
    DeltaB=(A*X-b)
    return (2/m)*np.array(A.T*DeltaB).flatten()

In [ ]:
# encontrar una forma iterativa de actualizar X para ir minimizando la funcion de costo L
x = np.zeros(Anumpy.shape[1])
epsilon = 2e-6
cost = []
N = 100
for it in range(N):
    x = x - epsilon*dLdx(x,Anumpy,bnumpy)
    cost.append(L(x,Anumpy,bnumpy))
sns.scatterplot(np.arange(N)+1,cost)
# plt.xscale('log')
# plt.yscale('log')

In [ ]:
L(np.array(X_opt).flatten(),Anumpy,bnumpy)

In [ ]:
np.array(X_opt).flatten()

In [ ]:
e=1e-10

In [ ]:
# %%timeit
res1 = sp.optimize.minimize(fun=L,jac=dLdx, x0=np.zeros(Anumpy.shape[1]), args = (Anumpy,bnumpy), method='Newton-CG', tol=e)
res1

In [ ]:
res2 = sp.optimize.minimize(fun=L,jac=dLdx, x0=np.zeros(Anumpy.shape[1]), args = (Anumpy,bnumpy), method='L-BFGS-B', tol=e)
res2

In [ ]:
L(res1.x,Anumpy,bnumpy)

In [ ]:
L(res2.x,Anumpy,bnumpy)

In [ ]:
L(np.array(X_opt).flatten(),Anumpy,bnumpy)

In [ ]:
calc_error(res2,e,names=[])

# Fit a funciones no lineales

In [ ]:
print("Nuestra caja experimental!")
a = 3/2
b = 4
c = -3
N=100
x = np.linspace(0.2,10,N)
y = a/(1+np.exp(c*(x-b)))
x1 = x + np.random.exponential(0.01,size=N)
y1 = y + np.random.normal(0,0.05,size=N)
x2 = x + np.random.normal(0,0.03,size=N)
y2 = y + np.random.exponential(0.05,size=N)

In [ ]:
sns.scatterplot(x1,y1)

In [ ]:
sns.scatterplot(x2,y2)

# ¿Cómo hacer el fit a la función?
$$f(x) = a\frac{1}{1+e^{bx+c}}$$

In [ ]:
#como encuentro yo a, b y c?
# cual seria nuestra funcion de costo
def f(parametros,x):
    return parametros[0]/(1+np.exp(parametros[1]*x+parametros[2]))
def Lfit(parametros,x,y):
    # L = promedio sobre todos los puntos (f(a,b,c;x)-y)^2
    # parametros np.array([a,b,c])
    deltaY=f(parametros,x) - y
    return np.dot(deltaY,deltaY)/len(y)

In [ ]:
print("Ajuste para el primer set: x1,y1")
e=1e-8
# ansatz: a=1,b=0,c=0
res1 = sp.optimize.minimize(fun=Lfit, x0=np.array([1,0,0]), args = (x1,y1), method='L-BFGS-B', tol=e)
res1

In [ ]:
y1_pred = f(res1.x,x1)
sns.scatterplot(x1,y1)
plt.plot(x1,y1_pred,color='r')

In [ ]:
r2 = 1-np.sum((y1_pred-y1)**2)/np.sum((y1-y1.mean())**2)
r2

In [ ]:
calc_error(res1,e,names=['a','b','c'])

# ¿Cómo hacer el fit a la función?
$$f(x) = a + b\tanh(cx+d)$$

In [ ]:
#como encuentro yo a, b y c?
# cual seria nuestra funcion de costo
def ftilde(parametros,x):
    return parametros[0]+parametros[1]*np.tanh(parametros[2]*x+parametros[3])
def Lfit(parametros,x,y):
    # L = promedio sobre todos los puntos (f(a,b,c,d;x)-y)^2
    # parametros np.array([a,b,c,d])
    deltaY=ftilde(parametros,x) - y
    return np.dot(deltaY,deltaY)/len(y)

In [ ]:
print("Ajuste para el primer set: x1,y1")
e=1e-8
# ansatz: a=0,b=1,c=0,d=0
res1 = sp.optimize.minimize(fun=Lfit, x0=np.array([0,1,0,0]), args = (x1,y1), method='L-BFGS-B', tol=e)
res1

In [ ]:
y1_pred = ftilde(res1.x,x1)
sns.scatterplot(x1,y1)
plt.plot(x1,y1_pred,color='r')

In [ ]:
r2 = 1-np.sum((y1_pred-y1)**2)/np.sum((y1-y1.mean())**2)
r2

In [ ]:
calc_error(res1,e,names=['a','b','c','d'])